# Movie Recommendation System
Problem statement: Given a search query, find $k$ movies similar to that movie using the available genres.  

Note:  
The search query consists of the title, and the vector containing it's genres.

---
## Step 1: Data
Data is taken from [Kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata/version/2#tmdb_5000_movies.csv).  
This data contains a lot of features which we don't need, thus we need to clean and modify this to our needs. 

In [18]:
import pandas as pd
df = pd.read_csv('tmdb_5000_movies.csv') # The size is 4803*20
display(df)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [19]:
genres = df['genres']
names = df['title']
popularity = df['popularity']
print(genres, '\n')
print(names, '\n')
print(popularity, '\n') # We will use this later to sort our results.

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object 

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                    

We notice that genres aren't the way we want, while titles and popularity are okay.  
Firstly, lets reduce the dataset.

In [20]:
df = df[['title','genres','popularity']]
display(df)

,title,genres,popularity
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",139.082615
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",107.376788
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",112.312950
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",43.926995
...,...,...,...
4798,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",14.269792
4799,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",0.642552
4800,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",1.444476
4801,Shanghai Calling,[],0.857008


Now, this looks much better.  
But still, our 'genres' column is pretty trash. Let's change that.

In [21]:
m = len(df) # The size(number of rows) of the dataframe
for i in range(m):
    listDict = eval(df['genres'][i])
    df['genres'][i] = [dictionary['name'] for dictionary in listDict]

display(df)

C:\Users\DELLL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,title,genres,popularity
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",150.437577
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",139.082615
2,Spectre,"[Action, Adventure, Crime]",107.376788
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",112.312950
4,John Carter,"[Action, Adventure, Science Fiction]",43.926995
...,...,...,...
4798,El Mariachi,"[Action, Crime, Thriller]",14.269792
4799,Newlyweds,"[Comedy, Romance]",0.642552
4800,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",1.444476
4801,Shanghai Calling,[],0.857008


This looks better.

---
# The distance function:
We now need to find a way to estimate the 'distance' between two genre lists.  
We can do the simple set intersection for this to identify the common genres.  

In [22]:
a = [1,2,4,5]
b = [2,4,8,6]
set(a).intersection(set(b))

{2, 4}

In [23]:
def distance(a, b):
    # Converting everything to lower case.
    a = [ai.lower() for ai in a] 
    b = [bi.lower() for bi in b]
    a = set(a)
    b = set(b)
    return len(a.intersection(b)) # The number of similarities determine the distance.

In [24]:
distance(["a","b","c"],["d","b","c","f"]) # Intersection

2

---
# The Algorithm:
Lets move on to the actual algorithm now.

In [25]:
def knn(query, k):
    DPI = [] # distances, popularity and indices.
    for i in range(m):
        d = distance(query, df['genres'][i])
        p = df['popularity'][i]
        DPI.append([d, p, i])
    recommendations = sorted(DPI)
    return recommendations[-k:] # k-nearest 'neighbours'

# Function that returns recommendations.
def getRec(genres, n):
    kRec = knn(genres, n) # n recommendations.
    indices = [kRec[i][2] for i in range(len(kRec))] # Automatically sorted(in asccending order) according to their popularity.
    recTitles = [df['title'][i] for i in indices]
    recTitles.reverse()
    for i in range(10):
        print(str(i+1) + ". " + recTitles[i])

In [38]:
# Enter your 
genres = ['Animation','Family'] # Enter the genres you want as a list.
n = 10 # Number of recommendations.

getRec(genres,n)

1. Minions
2. Big Hero 6
3. Frozen
4. Despicable Me 2
5. Inside Out
6. Brave
7. Spirited Away
8. Despicable Me
9. Monsters, Inc.
10. How to Train Your Dragon 2


That's it! We have built a simple recommendation system. There are a lot of things we can improve in this, particularly, the `distance` function which is now just a simple intersection of genres.

If you don't know what genres are available, take a look at this list:

- "id": 28,
  "name": "Action"
    
- "id": 12,
  "name": "Adventure"

- "id": 16,
  "name": "Animation"

- "id": 35,
  "name": "Comedy"

- "id": 80,
  "name": "Crime"

- "id": 99,
  "name": "Documentary"

- "id": 18,
  "name": "Drama"

- "id": 10751,
  "name": "Family"

- "id": 14,
  "name": "Fantasy"

- "id": 36,
  "name": "History"

- "id": 27,
  "name": "Horror"

- "id": 10402,
  "name": "Music"

- "id": 9648,
  "name": "Mystery"

- "id": 10749,
  "name": "Romance"

- "id": 878,
  "name": "Science Fiction"

- "id": 10770,
  "name": "TV Movie"

- "id": 53,
  "name": "Thriller"

- "id": 10752,
  "name": "War"

- "id": 37,
  "name": "Western"